# **Multiclass Fish Image Classification**

# **Libraries Used**

In [79]:
import os
import numpy as np

from sklearn.metrics import classification_report

from tensorflow.keras.preprocessing import image
from tensorflow.keras import models,layers
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2, InceptionV3, EfficientNetB0
from tensorflow.keras.optimizers import Adam

# **Data Loading**

Train Data:

In [ ]:
# Train Data path
train_data_path = "data/train"

# All Classes
classes = os.listdir(train_data_path)

# Ignore "Animal fish" and "Animal fish bass"
classes_filtered = [cls for cls in classes if cls.startswith("fish sea_food")]

# Keeping only Species name 
classes_new = [cls.replace("fish sea_food ", "") for cls in classes_filtered]
classes_final = [cls.replace("_", " ").title() for cls in classes_new]

# Rescaling
rescaled = image.ImageDataGenerator(rescale=1./255)

# Load Train Data
train_generator = rescaled.flow_from_directory(
    train_data_path,
    target_size=(224, 224),
    batch_size=32,
    classes=classes_filtered,
    class_mode="categorical"
)

Found 5099 images belonging to 9 classes.


Validation Data:

In [39]:
# Validation Data path
val_data_path = "data/val"

# Load Validation Data
val_generator = rescaled.flow_from_directory(
    val_data_path,
    target_size=(224, 224),
    batch_size=32,
    classes=classes_filtered,
    class_mode='categorical'
)

Found 895 images belonging to 9 classes.


Test Data:

In [40]:
# Test Data path
test_data_path = "data/test"

# Load Test Data
test_generator = rescaled.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=32,
    classes=classes_filtered,
    class_mode='categorical',
    shuffle=False
)

Found 2654 images belonging to 9 classes.


Label Mapping:

In [48]:
# Label Mapping 
index_labeled = {i: name for i, name in enumerate(classes_final)}

# Class Mapped
print("Class index mapping:", index_labeled, sep='\n')

Class index mapping:
{0: 'Black Sea Sprat', 1: 'Gilt Head Bream', 2: 'Hourse Mackerel', 3: 'Red Mullet', 4: 'Red Sea Bream', 5: 'Sea Bass', 6: 'Shrimp', 7: 'Striped Red Mullet', 8: 'Trout'}


# **CNN Model**

Model Structure:

In [42]:
cnn_model = models.Sequential([
    layers.Input(shape=(224,224,3)),
    layers.Conv2D(32,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(9,activation='softmax'),
])

Model Compiler:

In [43]:
cnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Model Train:

In [ ]:
cnn_trained = cnn_model.fit(
    train_generator, 
    epochs=5, 
    validation_data=val_generator
)

Epoch 1/5


c:\Users\vaisa\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


160/160 ━━━━━━━━━━━━━━━━━━━━ 57s 351ms/step - accuracy: 0.3832 - loss: 1.8659 - val_accuracy: 0.8067 - val_loss: 0.5700
Epoch 2/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 57s 355ms/step - accuracy: 0.8664 - loss: 0.3884 - val_accuracy: 0.9453 - val_loss: 0.1830
Epoch 3/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 56s 348ms/step - accuracy: 0.9601 - loss: 0.1249 - val_accuracy: 0.8961 - val_loss: 0.3233
Epoch 4/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 56s 348ms/step - accuracy: 0.9720 - loss: 0.0752 - val_accuracy: 0.9207 - val_loss: 0.2320
Epoch 5/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 55s 341ms/step - accuracy: 0.9869 - loss: 0.0387 - val_accuracy: 0.9609 - val_loss: 0.1265


Model Evaluate:

In [46]:
test_loss, test_accuracy = cnn_model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

c:\Users\vaisa\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 86ms/step - accuracy: 0.9551 - loss: 0.1299
Test Loss: 0.1506
Test Accuracy: 0.9506


Classification Report:

In [ ]:
y_pred_probs = cnn_model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes
report = classification_report(y_true, y_pred, target_names=classes_final)
print(report)

83/83 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step
                    precision    recall  f1-score   support

   Black Sea Sprat       0.93      0.99      0.96       298
   Gilt Head Bream       0.92      0.91      0.92       305
   Hourse Mackerel       0.95      0.92      0.94       286
        Red Mullet       0.98      0.99      0.99       291
     Red Sea Bream       0.93      0.97      0.95       273
          Sea Bass       0.95      0.92      0.94       327
            Shrimp       1.00      0.95      0.97       289
Striped Red Mullet       0.97      0.91      0.94       293
             Trout       0.92      1.00      0.96       292

          accuracy                           0.95      2654
         macro avg       0.95      0.95      0.95      2654
      weighted avg       0.95      0.95      0.95      2654



Image Test:

In [ ]:
def predict_fish_class(img_path, model, label_map):
    img = image.load_img(img_path, target_size=(224, 224))    
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    predicted_idx = np.argmax(predictions, axis=1)[0]
    predicted_name = label_map[predicted_idx]
    confidence = predictions[0][predicted_name]
    
    return predicted_name, confidence

In [55]:
img_path = r'data\test\fish sea_food sea_bass\0L8PRYEJDGPN.jpg'
predicted_class, confidence_score = predict_fish_class(img_path, cnn_model, index_labeled)

print(f"Predicted Fish Species: {predicted_class}")
print(f"Confidence: {confidence_score:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predicted Fish Species: Sea Bass
Confidence: 0.99


In [ ]:
img_path = r'data\test\fish sea_food red_mullet\0DTLG8H3NN5N.jpg'
predicted_class, confidence_score = predict_fish_class(img_path, cnn_model, index_labeled)

print(f"Predicted Fish Species: {predicted_class}")
print(f"Confidence: {confidence_score:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Fish Species: Red Mullet
Confidence: 1.00


Model Save:

In [49]:
cnn_model.save('First_Model_Good.keras')

# **Pre-Trained Models**

Models:

In [ ]:
pre_trained_models = [
    # {"name": "VGG16", "builder": VGG16, "preprocess": None},
    # {"name": "ResNet50", "builder": ResNet50, "preprocess": None},
    {"name": "InceptionV3", "builder": InceptionV3, "preprocess": None},
    {"name": "EfficientNetB0", "builder": EfficientNetB0, "preprocess": None},
    {"name": "MobileNetV2", "builder": MobileNetV2, "preprocess": None},
]

In [85]:
results = {}

for info in pre_trained_models:
    print(f"Training with {info['name']}")
    base_model = info["builder"](weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False

    # Build 
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        # layers.Dropout(0.3),
        layers.Dense(9, activation='softmax')
    ])

    # Compile
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train 
    history = model.fit(train_generator, epochs=5, validation_data=val_generator)

    # Evaluate
    test_loss, test_acc = model.evaluate(test_generator)
    print(f"{info['name']} Test Accuracy: {test_acc:.4f}")
    results[info['name']] = test_acc
    print()

Training with MobileNetV2
Epoch 1/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 48s 281ms/step - accuracy: 0.8188 - loss: 0.5871 - val_accuracy: 0.9888 - val_loss: 0.0477
Epoch 2/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 44s 275ms/step - accuracy: 0.9972 - loss: 0.0251 - val_accuracy: 0.9922 - val_loss: 0.0273
Epoch 3/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 44s 274ms/step - accuracy: 0.9999 - loss: 0.0103 - val_accuracy: 0.9978 - val_loss: 0.0153
Epoch 4/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 44s 275ms/step - accuracy: 1.0000 - loss: 0.0055 - val_accuracy: 0.9989 - val_loss: 0.0121
Epoch 5/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 44s 274ms/step - accuracy: 1.0000 - loss: 0.0034 - val_accuracy: 0.9989 - val_loss: 0.0105
83/83 ━━━━━━━━━━━━━━━━━━━━ 20s 239ms/step - accuracy: 0.9979 - loss: 0.0088
MobileNetV2 Test Accuracy: 0.9974

Training with InceptionV3
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 30s 0us/step
Epoch 1/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 69s 404ms/step - accuracy: 0.7529 - loss: 0.7623 - val_accuracy: 0.9743 - val_loss: 0.1080
Epoch

In [86]:
print("Summary of all models:")
for name, acc in results.items():
    print(f"{name}: {acc:.4f}")

Summary of all models:
MobileNetV2: 0.9974
InceptionV3: 0.9879
EfficientNetB0: 0.1100
